In [3]:
#import the libraries as needed
import os
import re
import time
import csv
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date, timedelta, datetime
from urllib.request import urlopen
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
! pip install pymysql

# Start of Important Code

In [4]:
#Reset list of URL's
correct_url = []
Url_list = []

In [5]:
#Reset Tables
ticker_data = []
company_data = []
title_data = []
date_data = []
time_data = []
quarter_data = []
company_info_table = []
Speech_table = []
QA_table = []
Comp_part_table = []
speech = []
speaker = []
QA_speech = []
QA_speaker = []
comp_parts = []
comp_pos = []
conf_parts = []
conf_pos = []
Conf_part_table = []
comp_ccid = []
speech_ccid = []
QA_ccid = []
comppart_ccid = []
confpart_ccid = []
speech_scriptid = []
QA_scriptid = []
comppart_partid = []
confpart_partid = []
test_list = []

In [10]:
def pullpageurls(siteurl):
    pages = np.arange(1,3,1)
    driver = webdriver.Chrome('./chromedriver')
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}

    #Pulling seeking alpha transcript search page
    for page in pages:
        page = siteurl+"/"+str(page)
        driver.get(page)
        site_html = driver.page_source
        soup_site = BeautifulSoup(site_html,'html.parser')
        sleep(random.randint(2,10))
        
        #Pulling transcript urls from seeking alpha transcript search page     
        url_begin = 'https://seekingalpha.com'
        site_content = soup_site.find_all('h3')
        #Extracting url and placing into a list
        for url in site_content:
            url_pull = url.find('a',class_ = 'dashboard-article-link')
            partial_url = url_pull.get('href')
            pageURL = url_begin+partial_url
            Url_list.append(pageURL)
            
    
    #Extracting only Url's with 2019 or 2020 in them
    
def pullpagedetails(Url_list):
    driver = webdriver.Chrome('./chromedriver')
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    counter = 0
    counter2 = 0
    counter3 = 0
    for i in Url_list:
        if ('2020' in i or '2019'in i) and ('earnings' in i):
            #Pulling relevant information from URL's

            #Header to mimic browser visit
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
            #Create requests.models.response object from URL
            driver.get(i)
            html = driver.page_source
            #Create the soup
            soup = BeautifulSoup(html, "lxml")
            call = soup.find('div',id = 'content-rail')
            
            #Testing if URL contains necessary elements
            test_list = []
            test = call.find_all('p',class_ = re.compile('p p*'))
            for c in test:
                x = c.text.strip()
                test_list.append(x)
            if ('Operator' in test_list) and ('Conference Call Participants' in test_list):
                
                #Basic Company Info Section

                #Create cc_id
                cc_id = counter


                #Find company ticker
                try:
                    ticker = call.find('a',class_ = 'ticker-link').text.split(" ")[-1]
                    ticker_data.append(ticker)
                except:
                    ticker_data.append('None')
                #Find company name
                try:
                    company = call.find('a', class_ = 'ticker-link').text.split(" ")[0:-1]
                    company = " ".join(company)
                    company_data.append(company)
                except:
                    company_data.append('None')
                #Find call title
                try:
                    title = call.find('h1').text
                    title_data.append(title)
                except:
                    title_data.append('None')
                #Find quarter/year
                try:
                    date_text = call.find('p',class_ = 'p p1').text.split(') ')[1]
                    quarter = date_text[0:7]
                    quarter_data.append(quarter)
                except:
                    quarter_data.append('None')
                #Find call date
                try:
                    date_string = date_text[date_text.index('Call')+5:].split("Call ")[0]
                    date = date_string[:date_string.index(":")-2].strip()
                    date_data.append(date)
                except:
                    date_data.append('None')
                #Find call time
                try:
                    time = date_string[date_string.index(":")-2:].strip()
                    time_data.append(time)
                except:
                    time_data.append('None')
                #Add CC ID
                comp_ccid.append(cc_id)

        #Speech and Company Participants Section

                bj = []
                indices = []
                convo = []
                #Pull speech script section
                script = call.find_all('p',class_ = re.compile('p p*'))
                #pull strong tags from speech script section
                for i in script:
                    if i.find('strong') is not None:
                        g = "strong"
                    else:
                        g = 0
                    bj.append(g)
                #Create list of index position of strong tags
                for j in range(len(bj)):
                    if bj[j] == 'strong':
                      indices.append(j)
                #Create list of text from speech script
                for k in script:
                    x = k.text.strip()
                    convo.append(x)

                #Create script ID & Participant ID
                script_id = counter2
                part_id = counter3

                #Create index objects to find positions of speech & participant sections
                conf_part_index = convo[convo.index('Conference Call Participants')+1:convo.index('Operator')]  
                speech_index = indices[0:indices.index(convo.index('Question-and-Answer Session'))]
                QA_index = indices[indices.index(convo.index('Question-and-Answer Session'))+1:]
                try:
                    comp_part_index = convo[convo.index('Company Participants')+1:convo.index('Conference Call Participants')]
                except:
                    comp_part_index = convo[convo.index('Corporate Participants')+1:convo.index('Conference Call Participants')]

                #Pulling info from speech section
                for i in range(2,len(speech_index)-1):
                    try:
                        z = convo[speech_index[i]+1:speech_index[i+1]]
                        speech.append(z)
                    except:
                        speech.append('None')
                    try:
                        g = convo[speech_index[i]]
                        speaker.append(g)
                    except:
                        speaker.append('None')
                    #Add CC ID & Script ID
                    speech_ccid.append(cc_id)
                    speech_scriptid.append(script_id)
                
                counter2 += 1
                script_id = counter2

                #Pulling info from Q&A section
                for i in range(0,len(QA_index)-1):
                    try:
                        q = convo[QA_index[i]+1:QA_index[i+1]]
                        QA_speech.append(q)
                    except:
                        QA_speech.append('None')
                    try:
                        f = convo[QA_index[i]]
                        QA_speaker.append(f)
                    except:
                        QA_speaker.append('None')
                    #Add CC ID & Script ID
                    QA_ccid.append(cc_id)
                    QA_scriptid.append(script_id)
                
                counter2 += 1
                script_id = counter2

                #Pulling company participants
                for i in comp_part_index:
                    x = i.split('-')
                    try:
                        a = x[0]
                        comp_parts.append(a)
                    except:
                        comp_parts.append('None')
                    try:
                        b = x[1]
                        comp_pos.append(b)
                    except:
                        comp_pos.append('None')
                    #Add CC ID & Part ID
                    comppart_ccid.append(cc_id)
                    comppart_partid.append(part_id)
                    
                    counter3 += 1
                    part_id = counter3

                #Pulling conference participants
                for i in conf_part_index:
                    x = i.split('-')
                    try:
                        a = x[0]
                        conf_parts.append(a)
                    except:
                        conf_parts.append('None')
                    try:
                        b = x[1]
                        conf_pos.append(b)
                    except:
                        conf_pos.append('None')
                    #Add CC ID & Part ID
                    confpart_ccid.append(cc_id)
                    confpart_partid.append(part_id)
                    
                    counter3 += 1
                    part_id = counter3
                    
                #Add to CC ID
                counter += 1

            else:
                continue

                
            
        #Pause in between trying each call transcript
        sleep(random.randint(2,10))

In [8]:
#Pull page urls
pullpageurls('https://seekingalpha.com/earnings/earnings-call-transcripts')

In [112]:
Url_list[400:500]

['https://seekingalpha.com/article/4390409-terrascend-corp-trssf-ceo-jason-ackerman-on-q3-2020-results-earnings-call-transcript',
 'https://seekingalpha.com/article/4390408-royal-mail-plc-roymf-ceo-stuart-simpson-on-half-year-2020minus-21-results-earnings-call',
 'https://seekingalpha.com/article/4390407-oaktree-specialty-lending-corporation-ocsl-ceo-armen-panossian-on-q4-2020-results-earnings',
 'https://seekingalpha.com/article/4390402-abbvie-inc-abbv-presents-wolfe-research-healthcare-conference-transcript',
 'https://seekingalpha.com/article/4390400-onewater-marine-inc-s-onew-ceo-austin-singleton-on-q4-2020-results-earnings-call-transcript',
 'https://seekingalpha.com/article/4390398-maximus-inc-mms-ceo-bruce-caswell-on-q4-2020-results-earnings-call-transcript',
 'https://seekingalpha.com/article/4390396-copa-holdings-s-cpa-ceo-pedro-heilbron-on-q3-2020-results-earnings-call-transcript',
 'https://seekingalpha.com/article/4390392-futu-holdings-limited-futu-ceo-leaf-li-on-q3-2020-re

In [11]:
#Pull page details
pullpagedetails(Url_list)

In [12]:
#Create tables

#Create company info table
company_info_table = pd.DataFrame({'CC ID':comp_ccid,'Tickers':ticker_data,'Company':company_data,'Call Title':title_data,'Fiscal Quarter':quarter_data,'Date':date_data,'Time':time_data})
#Create Speech table
Speech_table = pd.DataFrame({'CC ID':speech_ccid,'Script ID':speech_scriptid,'Speaker':speaker,'Speech':speech,'Section':'Speech'})
#Create Q&A table
QA_table = pd.DataFrame({'CC ID':QA_ccid,'Script ID':QA_scriptid,'Speaker':QA_speaker,'Speech':QA_speech,'Section':'Q&A'})
#Create Company participant table
Comp_part_table = pd.DataFrame({'CC ID':comppart_ccid,'Part ID':comppart_partid,'Participant':comp_parts,'Position':comp_pos,'Participant Type':'Company'})
#Create Conference participant table
Conf_part_table = pd.DataFrame({'CC ID':confpart_ccid,'Part ID':confpart_partid,'Participant':conf_parts,'Company':conf_pos,'Participant Type':'Conference'})

In [13]:
#Completed table of company info
company_info_table.head(50)

,CC ID,Tickers,Company,Call Title,Fiscal Quarter,Date,Time
0,0,(AFYA),Afya Limited,Afya Limited (AFYA) CEO Virgilio Gibbon on Q3 ...,Q3 2020,"December 4, 2020",10:00 AM ET
1,1,(LRCDF),Laurentian Bank of Canada,Laurentian Bank of Canada (LRCDF) CEO Rania Ll...,Q4 2020,"December 4, 2020",9:00 AM ET
2,2,(BIG),"Big Lots, Inc.","Big Lots, Inc. (BIG) CEO Bruce Thorn on Q3 202...",Q3 2020,"December 4, 2020",8:00 AM ET
3,3,(ZUMZ),Zumiez Inc.,Zumiez Inc. (ZUMZ) CEO Rick Brooks on Q3 2020 ...,Q3 2020,"December 3, 2020",5:00 PM ET
4,4,(COO),"The Cooper Companies, Inc.","The Cooper Companies, Inc. (COO) CEO Al White ...",Q4 2020,"December 3, 2020",5:00 PM ET
5,5,(OLLI),"Ollie's Bargain Outlet Holdings, Inc.","Ollie's Bargain Outlet Holdings, Inc. (OLLI) C...",Q3 2020,"December 3, 2020",4:30 PM ET
6,6,(ULTA),"Ulta Beauty, Inc.","Ulta Beauty, Inc.'s (ULTA) CEO Mary Dillon on ...",Q3 2020,"December 3, 2020",5:00 PM ET
7,7,(ATGN),"Altigen Communications, Inc.","Altigen Communications, Inc. (ATGN) CEO Jerry ...",Q4 2020,"December 3, 2020",5:00 PM ET
8,8,(TLYS),"Tilly's, Inc.","Tilly's, Inc. (TLYS) CEO Ed Thomas on Q3 2020 ...",Q3 2020,"December 3, 2020",4:30 PM ET
9,9,(SASDF),SAS AB (publ),SAS AB (publ) (SASDF) CEO Rickard Gustafson on...,(OTC:SA,None,4:30 PM ET


In [91]:
Speech_table.head(40)

,CC ID,Script ID,Speaker,Speech,Section
0,0,0,Operator,"[Good morning, ladies and gentlemen, and welco...",Speech
1,0,0,Renata Couto,[Good morning everyone. Thank you for joining ...,Speech
2,0,0,Virgilio Gibbon,"[Thank you, Renata, and thanks, everyone, for ...",Speech
3,1,2,Operator,"[Good day, bonjour and welcome to the Fourth Q...",Speech
4,1,2,Susan Cohen,"[Good morning, and thank you for joining us. T...",Speech
5,1,2,Rania Llewellyn,"[Thank you, Susan, and good morning. Today Lau...",Speech
6,1,2,Francois Laurin,"[Thank you, Rania, and good morning and bonjou...",Speech
7,2,4,Operator,"[Ladies and gentlemen, good morning, and welco...",Speech
8,2,4,Bruce Thorn,"[Thank you, and good morning, everyone. I am t...",Speech
9,3,6,Operator,"[Good afternoon, ladies and gentlemen. And wel...",Speech


In [92]:
QA_table.head(40)

,CC ID,Script ID,Speaker,Speech,Section
0,0,1,Operator,[Thank you. [Operator Instructions] Our first ...,Q&A
1,0,1,Marcelo Santos,"[Hi, good morning. Thanks for taking my questi...",Q&A
2,0,1,Virgilio Gibbon,"[Hi, Marcelo. Thank you for your question. I'l...",Q&A
3,0,1,Julio de Angeli,"[Hey, Marcelo. Hello, everyone. I hope everyon...",Q&A
4,0,1,Operator,[Thank you. Our next question comes from Susan...,Q&A
5,0,1,Susana Salaru,"[Hi, guys, good morning. Thank you for taking ...",Q&A
6,0,1,Julio de Angeli,"[Hey, Susana, Julio here. So in regards to the...",Q&A
7,0,1,Susana Salaru,"[Well, thank you, Julio. Just a follow-up on t...",Q&A
8,0,1,Julio de Angeli,"[Yeah. Well, the majority of the students, the...",Q&A
9,0,1,Operator,[Our next question comes from Fred Mendes with...,Q&A


In [93]:
Comp_part_table.head(40)

,CC ID,Part ID,Participant,Position,Participant Type
0,0,0,Renata Couto,Head of Investor Relations,Company
1,0,1,Virgilio Gibbon,Chief Executive Officer,Company
2,0,2,Luis Andre Blanco,Chief Financial Officer,Company
3,0,3,Julio de Angeli,Vice President of Innovation and Continuing M...,Company
4,1,9,Susan Cohen,"Director, IR",Company
5,1,10,Rania Llewellyn,President and CEO,Company
6,1,11,Francois Laurin,EVP and CFO,Company
7,1,12,Liam Mason,EVP and Chief Risk Officer,Company
8,2,18,Bruce Thorn,President and Chief Executive Officer,Company
9,2,19,Jonathan Ramsden,"Executive Vice President, Chief Financial and...",Company


In [94]:
Conf_part_table.head(40)

,CC ID,Part ID,Participant,Company,Participant Type
0,0,4,Marcelo Santos,JP Morgan,Conference
1,0,5,Susana Salaru,Itau,Conference
2,0,6,Fred Mendes,Bradesco,Conference
3,0,7,Mauricio Cepeda,Credit Suisse,Conference
4,0,8,Irma Sgarz,Goldman Sachs,Conference
5,1,13,Meny Grauman,Scotiabank,Conference
6,1,14,Gabriel Dechaine,National Bank Financial,Conference
7,1,15,Darko Mihelic,RBC Capital Markets,Conference
8,1,16,Doug Young,Desjardins Capital Markets,Conference
9,1,17,Sohrab Movahedi,BMO Capital Markets,Conference


In [ ]:
company_info_table.to_excel('C:\Users\rcjan\Documents\DataMgmtTables.xlsx', sheet_name='Comp_info', index = False)
speech_table.to_excel('C:\Users\rcjan\Documents\DataMgmtTables.xlsx', sheet_name='Speech_info', index = False)
QA_table.to_excel('C:\Users\rcjan\Documents\DataMgmtTables.xlsx', sheet_name='QA_info', index = False)
comp_part_table.to_excel('C:\Users\rcjan\Documents\DataMgmtTables.xlsx', sheet_name='Comp_part', index = False)
conf_part_table.to_excel('C:\Users\rcjan\Documents\DataMgmtTables.xlsx', sheet_name='Conf_part', index = False)

In [ ]:
import pymysql


conn = pymysql.connect(host='localhost', user='root',password='Asfhkl020102',db='sample')
curr = conn.cursor()
curr.execute("USE sample")

sql = "INSERT INTO `pets` (`name`, `owner`, `species`,`sex`) VALUES (%s, %s, %s, %s)"
val = list1
curr.executemany(sql, val)
sql1 = "INSERT INTO `babies` (`name`, `owner`, `species`,`sex`) VALUES (%s, %s, %s, %s)"
val1 = list2
curr.executemany(sql1, val1)
curr.execute("SELECT * FROM pets")
curr.execute("SELECT * FROM babies")
print(curr.fetchall())
conn.commit()
curr.close()
conn.close()

In [ ]:
# Create pets dataframe

# name, owner, species, sex

import pandas as pd
import numpy as np

d = {'name': ["Zuile", "Diamond"], 'owner': ["Alyce", "Thomas"],'species': ["dog", "dog"], 'sex': ["F", "F"] }
df = pd.DataFrame(data=d)
print(df)

# Covert to list

list1 = df.values.tolist()
print(list1)

In [ ]:
# Create babies dataframe

d2 = {'name': ["Bubbles", "Pepper"], 'owner': ["LeeLee", "Ebony"],'species': ["baby", "baby"], 'sex': ["F", "M"] }
df2 = pd.DataFrame(data=d2)
print(df2)

# Covert to list

list2 = df2.values.tolist()
print(list2)